### 重叠聚类图

#### 文件保存-后续网络分析需要

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

# 路径配置
current_dir = Path.cwd()
data_dir = current_dir if (current_dir / "3-1-L2_Policy_Clustering_Breadth.csv").exists() else current_dir.parent / "data"

def compute_overlap_matrix(input_path: Path) -> pd.DataFrame:
    """
    计算国家间在相同政策下聚类ID一致的重叠数量矩阵。

    Args:
        input_path (Path): 输入CSV文件路径。

    Returns:
        pd.DataFrame: 索引和列均为国家名的对称矩阵（共现次数）。
    """
    df = pd.read_csv(input_path, encoding='utf-8-sig')

    # 统一列名
    if 'L2政策' not in df.columns and 'L2政策中文名' in df.columns:
        df.rename(columns={'L2政策中文名': 'L2政策'}, inplace=True)

    # 提取核心字段 (政策, 国家, 聚类ID)
    data = df[['L2政策', '国家', '聚类ID']].drop_duplicates()
    data.columns = ['P', 'C', 'G']  # 简化列名引用

    policies = sorted(data['P'].unique())
    countries = sorted(data['C'].unique())
    n = len(countries)

    # 构建查询表: {政策: {国家: 聚类ID}}
    policy_map = {
        p: dict(zip(
            data[data['P'] == p]['C'],
            data[data['P'] == p]['G']
        )) for p in policies
    }

    # 计算矩阵
    matrix = np.zeros((n, n), dtype=int)
    for i, c1 in enumerate(countries):
        for j in range(i, n):
            c2 = countries[j]
            # 统计两者拥有相同聚类ID的政策数量
            count = sum(
                1 for p in policies
                if c1 in policy_map[p] and c2 in policy_map[p]
                and policy_map[p][c1] == policy_map[p][c2]
            )
            matrix[i, j] = matrix[j, i] = count

    return pd.DataFrame(matrix, index=countries, columns=countries)

if __name__ == "__main__":
    tasks = [
        ("3-1-L2_Policy_Clustering_Breadth.csv", "_Breadth"),
        ("3-1-L2_Policy_Clustering_Intensity.csv", "_Intensity")
    ]

    for filename, suffix in tasks:
        file_path = data_dir / filename
        if file_path.exists():
            # 计算并保存
            matrix_df = compute_overlap_matrix(file_path)
            output_path = data_dir / f"4-1-overlapping_cluster_heatmap{suffix}.csv"
            matrix_df.to_csv(output_path, encoding='utf-8-sig')

#### 分部门（生成overlap文件）

In [3]:
import pandas as pd
import numpy as np
import json
from pathlib import Path

# 1. 简化的环境配置
# 优先查找 ../data (假设脚本在 code 目录)，否则用当前目录
data_dir = Path.cwd().parent / "data"
if not (data_dir / "config_mappings.json").exists():
    data_dir = Path.cwd()

# 2. 加载 L2->L1 映射
with open(data_dir / "config_mappings.json", 'r', encoding='utf-8') as f:
    # 提取 json 中的 {"L2": ..., "L1": ...} 结构
    l2_to_l1_map = {item['L2']: item['L1'] for item in json.load(f).get('level_mapping', {}).values()}

def compute_and_save(df_input, output_path):
    """核心计算函数：计算共现矩阵并保存"""
    # 提取必要列去重
    data = df_input[['L2政策', '国家', '聚类ID']].drop_duplicates()
    
    # 准备国家索引，确保矩阵行列顺序固定
    countries = sorted(data['国家'].unique())
    c_idx = {c: i for i, c in enumerate(countries)}
    n = len(countries)
    matrix = np.zeros((n, n), dtype=int)
    
    # 按政策遍历，计算每项政策下的贡献
    for policy, group in data.groupby('L2政策'):
        # 将该政策下的国家按聚类ID分组: {ID: [Country1, Country2...]}
        clusters = {}
        for _, row in group.iterrows():
            clusters.setdefault(row['聚类ID'], []).append(row['国家'])
            
        # 同一个聚类ID下的所有国家，两两重叠度+1
        for members in clusters.values():
            idxs = [c_idx[m] for m in members]
            for i in range(len(idxs)):
                for j in range(i, len(idxs)):
                    r, c = idxs[i], idxs[j]
                    matrix[r, c] += 1
                    if r != c: matrix[c, r] += 1

    # 保存
    pd.DataFrame(matrix, index=countries, columns=countries).to_csv(output_path, encoding='utf-8-sig')
    print(f"已保存: {output_path.name}")

# 3. 主流程
tasks = [("3-1-L2_Policy_Clustering_Breadth.csv", "Breadth"), 
         ("3-1-L2_Policy_Clustering_Intensity.csv", "Intensity")]

for fname, folder_name in tasks:
    fpath = data_dir / fname
    if not fpath.exists(): continue
    
    # 读取数据
    df = pd.read_csv(fpath, encoding='utf-8-sig')
    if 'L2政策' not in df.columns: df.rename(columns={'L2政策中文名': 'L2政策'}, inplace=True)
    
    # 创建输出目录
    out_dir = data_dir / "4-1-overlapping_cluster_heatmap" / folder_name
    out_dir.mkdir(parents=True, exist_ok=True)
    
    # Task A: 计算全量矩阵 (所有政策)
    compute_and_save(df, out_dir / "Y_All.csv")
    
    # Task B: 按 L1 分组计算
    df['L1_Category'] = df['L2政策'].map(l2_to_l1_map)
    for l1_name, sub_df in df.groupby('L1_Category'):
        safe_name = l1_name.replace('/', '_').replace('\\', '_') # 防止路径报错
        compute_and_save(sub_df, out_dir / f"Y_{safe_name}.csv")

已保存: Y_All.csv
已保存: Y_Commitment-based.csv
已保存: Y_Incentive-based.csv
已保存: Y_Regulatory.csv
已保存: Y_Research and Development (R&D).csv
已保存: Y_All.csv
已保存: Y_Commitment-based.csv
已保存: Y_Incentive-based.csv
已保存: Y_Regulatory.csv
已保存: Y_Research and Development (R&D).csv


#### 相同布局分布

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from matplotlib.font_manager import FontProperties
import seaborn as sns
from scipy.cluster.hierarchy import linkage, fcluster
from pathlib import Path
import colorsys
import warnings

warnings.filterwarnings('ignore')

# ==========================================
# 1. 基础配置 (Times New Roman + 极致粗度)
# ==========================================
current_dir = Path.cwd()
data_dir = current_dir if (current_dir / "3-1-L2_Policy_Clustering_Breadth.csv").exists() else current_dir.parent / "data"

def get_times_black_font():
    """锁定新罗马并强制开启极致加粗"""
    return FontProperties(family='Times New Roman', size=26, weight='black')

T_BLACK = get_times_black_font()

# 全局环境设置
plt.rcParams.update({
    'font.family': 'serif',
    'font.serif': ['Times New Roman'],
    'font.weight': 'black',
    'axes.labelweight': 'black',
    'axes.titleweight': 'black',
    'axes.unicode_minus': False,
    'figure.dpi': 300
})

sns.set_theme(style="white")

# 关键：Seaborn 经常会把 axisbelow 打开，导致刻度线/网格永远画在下面
plt.rcParams['axes.axisbelow'] = False

def lighten_color_slightly(hex_color: str) -> str:
    r, g, b = plt.cm.colors.to_rgb(hex_color)
    h, l, s = colorsys.rgb_to_hls(r, g, b)
    return plt.cm.colors.to_hex(colorsys.hls_to_rgb(h, min(1.0, l + 0.12), s * 0.88))

def get_palette(k: int) -> list:
    colors = ['#1f77b4', '#2ca02c', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
    return (colors * (k // len(colors) + 1))[:k]

# ==========================================
# 2. 数据处理辅助函数
# ==========================================
def build_overlap_matrix(input_file: Path) -> (pd.DataFrame, int):
    if not input_file.exists():
        raise FileNotFoundError(f"找不到文件: {input_file}")

    df = pd.read_csv(input_file, encoding='utf-8-sig')

    if 'L2政策' not in df.columns and 'L2政策中文名' in df.columns:
        df.rename(columns={'L2政策中文名': 'L2政策'}, inplace=True)

    def safe_col(d, name):
        return d[name].iloc[:, 0] if isinstance(d[name], pd.DataFrame) else d[name]

    df_clean = pd.DataFrame({
        'P': safe_col(df, 'L2政策').astype(str),
        'C': safe_col(df, '国家').astype(str),
        'G': safe_col(df, '聚类ID')
    }).drop_duplicates()

    policies = sorted(df_clean['P'].unique())
    countries = sorted(df_clean['C'].unique())

    res_map = {p: dict(zip(df_clean[df_clean['P'] == p]['C'],
                           df_clean[df_clean['P'] == p]['G'])) for p in policies}

    n = len(countries)
    matrix = np.zeros((n, n), dtype=int)
    for i, c1 in enumerate(countries):
        for j in range(i, n):
            c2 = countries[j]
            common = sum(
                1 for p in policies
                if c1 in res_map[p] and c2 in res_map[p]
                and (res_map[p][c1] == res_map[p][c2] if i != j else True)
            )
            matrix[i, j] = matrix[j, i] = common

    return pd.DataFrame(matrix, index=countries, columns=countries), len(policies)

# ==========================================
# 3. 核心绘图逻辑
# ==========================================
def draw_heatmap(data_df: pd.DataFrame, Z_master, row_colors, common_countries,
                 filename: str, title: str, v_max: int):

    # 画布大小调整
    size = max(18, len(common_countries) * 0.72)

    g = sns.clustermap(
        data_df, row_linkage=Z_master, col_linkage=Z_master,
        cmap='RdYlBu_r', row_colors=row_colors, col_colors=row_colors,
        dendrogram_ratio=0.1, linewidths=0.5, figsize=(size, size),
        annot=True, fmt='d', vmin=0, vmax=v_max,
        # 内部数字极致加粗
        annot_kws={'fontsize': 28, 'weight': 'black', 'fontproperties': T_BLACK},
        cbar_kws={'ticks': [0, 5, 10, 15, 20]},
        tree_kws={'linewidths': 6}
    )

    # =========================================================
    # A. 色条位置：根据热图 bbox 动态放到右侧（解决“太往左”）
    # =========================================================
    hm_pos = g.ax_heatmap.get_position()
    pad = 0.04        # 与热图的水平间距
    cbar_w = 0.018    # 色条宽度
    cbar_h = hm_pos.height * 0.78
    cbar_y = hm_pos.y0 + hm_pos.height * 0.11

    g.cax.set_position([hm_pos.x1 + pad, cbar_y, cbar_w, cbar_h])
    g.cax.yaxis.set_label_position('right')  # 标题放右侧更舒服

    # =========================================================
    # B. 关键修复：刻度线/双向指针压在色块之上（非 zorder 玄学）
    # =========================================================
    # 1) 禁止 axisbelow（否则 ticks 永远在下面）
    g.cax.set_axisbelow(False)

    # 2) 把色条色块的 collection 压到更底层
    for coll in g.cax.collections:
        coll.set_zorder(0)

    # 3) 双侧刻度线
    g.cax.yaxis.set_ticks_position('both')

    # 4) 刻度线“别太长太粗”，但要能盖住色块边缘
    g.cax.tick_params(
        axis='y',
        direction='in',   # 内向指针
        length=12,
        width=3,
        colors='black',
        left=True,
        right=True,
        pad=6
    )

    # 5) 让 tickline 真正到最上层，并取消裁剪（避免被色条区域裁掉）
    for t in g.cax.yaxis.get_major_ticks():
        for ln in (t.tick1line, t.tick2line):
            ln.set_zorder(10)
            ln.set_clip_on(False)

    # tick label 也提一下
    for lab in g.cax.get_yticklabels():
        lab.set_zorder(11)

    # 色条标题和刻度文字
    g.cax.set_ylabel(f'Overlap Count ({title})', fontproperties=T_BLACK,
                     fontsize=32, weight='black', labelpad=20)
    plt.setp(g.cax.get_yticklabels(), fontproperties=T_BLACK, fontsize=28, weight='black')

    # =========================================================
    # C. 坐标轴标签 (X/Y轴)
    # =========================================================
    for lab in g.ax_heatmap.get_xticklabels():
        lab.set_fontproperties(T_BLACK)
        lab.set_rotation(45)
        lab.set_ha('right')

    for lab in g.ax_heatmap.get_yticklabels():
        lab.set_fontproperties(T_BLACK)
        lab.set_rotation(0)

    # =========================================================
    # D. 热图边框极致加粗
    # =========================================================
    ax = g.ax_heatmap
    for spine in ax.spines.values():
        spine.set_visible(True)
        spine.set_linewidth(4)
        spine.set_edgecolor('black')

    # 保存
    plt.savefig(data_dir / filename, dpi=300, bbox_inches='tight')
    print(f"✅ 保存成功: {filename}")
    plt.close('all')

def plot_intensity_based_comparison(breadth_file: Path, intensity_file: Path) -> None:
    df_b, max_b = build_overlap_matrix(breadth_file)
    df_i, max_i = build_overlap_matrix(intensity_file)

    common_countries = sorted(list(set(df_b.index) & set(df_i.index)))
    df_b, df_i = df_b.loc[common_countries, common_countries], df_i.loc[common_countries, common_countries]

    # 层次聚类排序
    Z_master = linkage(df_i.values, method='ward')
    weights = df_i.sum(axis=1).values
    w_map = {i: v for i, v in enumerate(weights)}
    for i, (c1, c2, d, cnt) in enumerate(Z_master):
        c1, c2 = int(c1), int(c2)
        if w_map[c1] < w_map[c2]:
            Z_master[i, 0], Z_master[i, 1] = c2, c1
        w_map[len(weights) + i] = w_map[c1] + w_map[c2]

    k = min(11, len(common_countries))
    labels = fcluster(Z_master, t=k, criterion='maxclust') - 1
    row_colors = pd.Series([lighten_color_slightly(get_palette(k)[l]) for l in labels], index=common_countries)

    # 绘制两张图
    draw_heatmap(df_i, Z_master, row_colors, common_countries,
                 "4-1-aligned_by_intensity_INTENSITY.png", "Intensity", max_i)
    draw_heatmap(df_b, Z_master, row_colors, common_countries,
                 "4-1-aligned_by_intensity_BREADTH.png", "Breadth", max_b)

if __name__ == "__main__":
    plot_intensity_based_comparison(
        data_dir / "3-1-L2_Policy_Clustering_Breadth.csv",
        data_dir / "3-1-L2_Policy_Clustering_Intensity.csv"
    )


✅ 保存成功: 4-1-aligned_by_intensity_INTENSITY.png
✅ 保存成功: 4-1-aligned_by_intensity_BREADTH.png


#### 重心左上

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
import seaborn as sns
from scipy.cluster.hierarchy import linkage, fcluster, leaves_list
from pathlib import Path
import colorsys
import warnings

warnings.filterwarnings('ignore')

# ==========================================
# 1. 基础配置
# ==========================================
current_dir = Path.cwd()
data_dir = current_dir if (current_dir / "3-1-L2_Policy_Clustering_Breadth.csv").exists() else current_dir.parent / "data"

def get_times_black_font():
    return FontProperties(family='Times New Roman', size=26, weight='black')

T_BLACK = get_times_black_font()

plt.rcParams.update({
    'font.family': 'serif',
    'font.serif': ['Times New Roman'],
    'font.weight': 'black',
    'axes.labelweight': 'black',
    'axes.titleweight': 'black',
    'axes.unicode_minus': False,
    'figure.dpi': 300
})

sns.set_theme(style="white")
plt.rcParams['axes.axisbelow'] = False

def lighten_color_slightly(hex_color: str) -> str:
    r, g, b = plt.cm.colors.to_rgb(hex_color)
    h, l, s = colorsys.rgb_to_hls(r, g, b)
    return plt.cm.colors.to_hex(colorsys.hls_to_rgb(h, min(1.0, l + 0.12), s * 0.88))

def get_palette(k: int) -> list:
    colors = ['#1f77b4', '#2ca02c', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
    return (colors * (k // len(colors) + 1))[:k]

# ==========================================
# 2. 数据处理辅助函数
# ==========================================
def build_overlap_matrix(input_file: Path) -> (pd.DataFrame, int):
    if not input_file.exists():
        raise FileNotFoundError(f"找不到文件: {input_file}")

    df = pd.read_csv(input_file, encoding='utf-8-sig')

    if 'L2政策' not in df.columns and 'L2政策中文名' in df.columns:
        df.rename(columns={'L2政策中文名': 'L2政策'}, inplace=True)

    def safe_col(d, name):
        return d[name].iloc[:, 0] if isinstance(d[name], pd.DataFrame) else d[name]

    df_clean = pd.DataFrame({
        'P': safe_col(df, 'L2政策').astype(str),
        'C': safe_col(df, '国家').astype(str),
        'G': safe_col(df, '聚类ID')
    }).drop_duplicates()

    policies = sorted(df_clean['P'].unique())
    countries = sorted(df_clean['C'].unique())

    res_map = {p: dict(zip(df_clean[df_clean['P'] == p]['C'],
                           df_clean[df_clean['P'] == p]['G'])) for p in policies}

    n = len(countries)
    matrix = np.zeros((n, n), dtype=int)
    for i, c1 in enumerate(countries):
        for j in range(i, n):
            c2 = countries[j]
            common = sum(
                1 for p in policies
                if c1 in res_map[p] and c2 in res_map[p]
                and (res_map[p][c1] == res_map[p][c2] if i != j else True)
            )
            matrix[i, j] = matrix[j, i] = common

    return pd.DataFrame(matrix, index=countries, columns=countries), len(policies)

# ==========================================
# 3. 核心绘图逻辑
# ==========================================
def draw_heatmap(data_df: pd.DataFrame, Z_linkage, row_colors, common_countries,
                 filename: str, title: str, v_max: int):

    size = max(18, len(common_countries) * 0.72)

    g = sns.clustermap(
        data_df, 
        row_linkage=Z_linkage, 
        col_linkage=Z_linkage, 
        cmap='RdYlBu_r', 
        row_colors=row_colors, 
        col_colors=row_colors,
        dendrogram_ratio=0.1, linewidths=0.5, figsize=(size, size),
        annot=True, fmt='d', vmin=0, vmax=v_max,
        annot_kws={'fontsize': 28, 'weight': 'black', 'fontproperties': T_BLACK},
        cbar_kws={'ticks': [0, 5, 10, 15, 20]},
        tree_kws={'linewidths': 6}
    )

    # 色条和轴标签调整
    hm_pos = g.ax_heatmap.get_position()
    pad = 0.04; cbar_w = 0.018; cbar_h = hm_pos.height * 0.78
    cbar_y = hm_pos.y0 + hm_pos.height * 0.11
    g.cax.set_position([hm_pos.x1 + pad, cbar_y, cbar_w, cbar_h])
    g.cax.yaxis.set_label_position('right')

    g.cax.set_axisbelow(False)
    for coll in g.cax.collections: coll.set_zorder(0)
    g.cax.yaxis.set_ticks_position('both')
    g.cax.tick_params(axis='y', direction='in', length=12, width=3, colors='black', left=True, right=True, pad=6)

    for t in g.cax.yaxis.get_major_ticks():
        for ln in (t.tick1line, t.tick2line):
            ln.set_zorder(10); ln.set_clip_on(False)
    for lab in g.cax.get_yticklabels(): lab.set_zorder(11)

    g.cax.set_ylabel(f'Overlap Count ({title})', fontproperties=T_BLACK, fontsize=32, weight='black', labelpad=20)
    plt.setp(g.cax.get_yticklabels(), fontproperties=T_BLACK, fontsize=28, weight='black')

    for lab in g.ax_heatmap.get_xticklabels():
        lab.set_fontproperties(T_BLACK); lab.set_rotation(45); lab.set_ha('right')
    for lab in g.ax_heatmap.get_yticklabels():
        lab.set_fontproperties(T_BLACK); lab.set_rotation(0)

    ax = g.ax_heatmap
    for spine in ax.spines.values():
        spine.set_visible(True); spine.set_linewidth(4); spine.set_edgecolor('black')

    plt.savefig(data_dir / filename, dpi=300, bbox_inches='tight')
    print(f"✅ 保存成功: {filename}")
    plt.close('all')

# ==========================================
# 4. 关键：强制左侧排序 (Left-Heavy Sort)
# ==========================================
def compute_left_heavy_clustering(df: pd.DataFrame, k: int):
    """
    独立计算聚类，并进行物理检查：
    如果聚类结果显示重心在右侧（大数值国家在后面），则强制镜像翻转，确保大数值在左侧。
    """
    # 1. 计算基础聚类
    Z = linkage(df.values, method='ward')
    weights = df.sum(axis=1).values
    
    # 2. 初步排序：尝试将大权重排在左子节点 (Index 0)
    #    (虽然这步可能不够，后面的步骤4会做最终兜底)
    w_map = {i: v for i, v in enumerate(weights)}
    for i, (c1, c2, d, cnt) in enumerate(Z):
        c1, c2 = int(c1), int(c2)
        if w_map[c1] < w_map[c2]: # 如果左 < 右，则交换，让左边大
            Z[i, 0], Z[i, 1] = c2, c1
        w_map[len(weights) + i] = w_map[c1] + w_map[c2]

    # 3. 获取叶子节点顺序 (Leaves Order)
    current_leaves = leaves_list(Z)
    
    # 4. 【核心修正】重心检测与强制翻转
    #    计算左半边和右半边的总权重
    mid_point = len(current_leaves) // 2
    left_mass = weights[current_leaves[:mid_point]].sum()
    right_mass = weights[current_leaves[mid_point:]].sum()
    
    print(f"   [重心检测] 左侧权重: {left_mass:.0f} | 右侧权重: {right_mass:.0f}")

    if right_mass > left_mass:
        print("   >>> ⚠️ 检测到重心在右侧，正在强制执行全树镜像翻转 (Mirror Flip)...")
        # 翻转 Z 矩阵的所有左右子节点引用，这会直接导致 dendrogram 镜像翻转
        Z[:, [0, 1]] = Z[:, [1, 0]]
        print("   >>> 翻转完成，大数值已移至左侧。")
    else:
        print("   >>> ✅ 重心已在左侧，无需调整。")

    # 5. 生成颜色
    labels = fcluster(Z, t=k, criterion='maxclust') - 1
    palette = get_palette(k)
    row_colors = pd.Series(
        [lighten_color_slightly(palette[l % len(palette)]) for l in labels], 
        index=df.index
    )

    return Z, row_colors

# ==========================================
# 5. 主流程
# ==========================================
def plot_independent_comparison(breadth_file: Path, intensity_file: Path) -> None:
    # 1. 读取数据
    df_b, max_b = build_overlap_matrix(breadth_file)
    df_i, max_i = build_overlap_matrix(intensity_file)

    # 2. 统一国家范围
    common_countries = sorted(list(set(df_b.index) & set(df_i.index)))
    df_b = df_b.loc[common_countries, common_countries]
    df_i = df_i.loc[common_countries, common_countries]
    
    k = min(11, len(common_countries))

    # 3. 处理 Intensity
    print("\n--------- 处理 Intensity ---------")
    Z_i, colors_i = compute_left_heavy_clustering(df_i, k)
    draw_heatmap(
        df_i, Z_i, colors_i, common_countries,
        "4-1-independent_ForceLeft_INTENSITY.png", "Intensity", max_i
    )

    # 4. 处理 Breadth
    print("\n--------- 处理 Breadth ---------")
    Z_b, colors_b = compute_left_heavy_clustering(df_b, k)
    draw_heatmap(
        df_b, Z_b, colors_b, common_countries,
        "4-1-independent_ForceLeft_BREADTH.png", "Breadth", max_b
    )

if __name__ == "__main__":
    plot_independent_comparison(
        data_dir / "3-1-L2_Policy_Clustering_Breadth.csv",
        data_dir / "3-1-L2_Policy_Clustering_Intensity.csv"
    )


--------- 处理 Intensity ---------
   [重心检测] 左侧权重: 8066 | 右侧权重: 7731
   >>> ✅ 重心已在左侧，无需调整。
✅ 保存成功: 4-1-independent_ForceLeft_INTENSITY.png

--------- 处理 Breadth ---------
   [重心检测] 左侧权重: 6977 | 右侧权重: 8088
   >>> ⚠️ 检测到重心在右侧，正在强制执行全树镜像翻转 (Mirror Flip)...
   >>> 翻转完成，大数值已移至左侧。
✅ 保存成功: 4-1-independent_ForceLeft_BREADTH.png


In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
import seaborn as sns
from scipy.cluster.hierarchy import linkage, fcluster, leaves_list
from pathlib import Path
import colorsys
import warnings

warnings.filterwarnings('ignore')

# ==========================================
# 1. 基础配置
# ==========================================
current_dir = Path.cwd()
data_dir = current_dir if (current_dir / "3-1-L2_Policy_Clustering_Breadth.csv").exists() else current_dir.parent / "data"


def get_times_black_font() -> FontProperties:
    return FontProperties(family='Times New Roman', size=26, weight='black')


T_BLACK = get_times_black_font()

plt.rcParams.update({
    'font.family': 'serif',
    'font.serif': ['Times New Roman'],
    'font.weight': 'black',
    'axes.labelweight': 'black',
    'axes.titleweight': 'black',
    'axes.unicode_minus': False,
    'figure.dpi': 300
})

sns.set_theme(style="white")
plt.rcParams['axes.axisbelow'] = False


def lighten_color_slightly(hex_color: str) -> str:
    r, g, b = plt.cm.colors.to_rgb(hex_color)
    h, l, s = colorsys.rgb_to_hls(r, g, b)
    return plt.cm.colors.to_hex(
        colorsys.hls_to_rgb(h, min(1.0, l + 0.12), s * 0.88)
    )


def get_palette(k: int) -> list:
    colors = [
        '#1f77b4', '#2ca02c', '#9467bd', '#8c564b',
        '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'
    ]
    return (colors * (k // len(colors) + 1))[:k]


# ==========================================
# 2. 数据处理
# ==========================================
def build_overlap_matrix(input_file: Path) -> tuple[pd.DataFrame, int]:
    df = pd.read_csv(input_file, encoding='utf-8-sig')

    if 'L2政策' not in df.columns and 'L2政策中文名' in df.columns:
        df.rename(columns={'L2政策中文名': 'L2政策'}, inplace=True)

    def safe_col(d: pd.DataFrame, name: str) -> pd.Series:
        return d[name].iloc[:, 0] if isinstance(d[name], pd.DataFrame) else d[name]

    df_clean = pd.DataFrame({
        'P': safe_col(df, 'L2政策').astype(str),
        'C': safe_col(df, '国家').astype(str),
        'G': safe_col(df, '聚类ID')
    }).drop_duplicates()

    policies = sorted(df_clean['P'].unique())
    countries = sorted(df_clean['C'].unique())

    res_map = {
        p: dict(zip(
            df_clean[df_clean['P'] == p]['C'],
            df_clean[df_clean['P'] == p]['G']
        ))
        for p in policies
    }

    n = len(countries)
    matrix = np.zeros((n, n), dtype=int)

    for i, c1 in enumerate(countries):
        for j in range(i, n):
            c2 = countries[j]
            common = sum(
                1 for p in policies
                if c1 in res_map[p]
                and c2 in res_map[p]
                and (res_map[p][c1] == res_map[p][c2] if i != j else True)
            )
            matrix[i, j] = matrix[j, i] = common

    return pd.DataFrame(matrix, index=countries, columns=countries), len(policies)


# ==========================================
# 3. 绘图
# ==========================================
def draw_heatmap(
    data_df: pd.DataFrame,
    Z_linkage,
    row_colors: pd.Series,
    filename: str,
    title: str,
    v_max: int
) -> None:

    size = max(18, len(data_df) * 0.72)

    g = sns.clustermap(
        data_df,
        row_linkage=Z_linkage,
        col_linkage=Z_linkage,
        cmap='RdYlBu_r',
        row_colors=row_colors,
        col_colors=row_colors,
        dendrogram_ratio=0.1,
        linewidths=0.5,
        figsize=(size, size),
        annot=True,
        fmt='d',
        vmin=0,
        vmax=v_max,
        annot_kws={'fontsize': 28, 'weight': 'black', 'fontproperties': T_BLACK},
        tree_kws={'linewidths': 6}
    )

    save_path = data_dir / filename
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close('all')

    print(f"✅ 保存成功: {filename}")


# ==========================================
# 4. 强制左下角重心聚类
# ==========================================
def compute_force_left_clustering(
    df: pd.DataFrame,
    k: int
) -> tuple[np.ndarray, pd.Series, dict]:

    Z = linkage(df.values, method='ward')
    weights = df.sum(axis=1).values

    # 局部：保证大权重在左子树
    w_map = {i: v for i, v in enumerate(weights)}
    for i, (c1, c2, _, _) in enumerate(Z):
        c1, c2 = int(c1), int(c2)
        if w_map[c1] < w_map[c2]:
            Z[i, 0], Z[i, 1] = c2, c1
        w_map[len(weights) + i] = w_map[c1] + w_map[c2]

    leaves = leaves_list(Z)
    mid = len(leaves) // 2
    left_weight = int(weights[leaves[:mid]].sum())
    right_weight = int(weights[leaves[mid:]].sum())

    flipped = False
    if right_weight > left_weight:
        Z[:, [0, 1]] = Z[:, [1, 0]]
        flipped = True

    labels = fcluster(Z, t=k, criterion='maxclust') - 1
    palette = get_palette(k)
    row_colors = pd.Series(
        [lighten_color_slightly(palette[l % len(palette)]) for l in labels],
        index=df.index
    )

    info = {
        "left_weight": left_weight,
        "right_weight": right_weight,
        "flipped": flipped
    }

    return Z, row_colors, info


# ==========================================
# 5. 主流程
# ==========================================
def plot_independent_comparison(
    breadth_file: Path,
    intensity_file: Path
) -> None:

    df_b, max_b = build_overlap_matrix(breadth_file)
    df_i, max_i = build_overlap_matrix(intensity_file)

    common_countries = sorted(set(df_b.index) & set(df_i.index))
    df_b = df_b.loc[common_countries, common_countries]
    df_i = df_i.loc[common_countries, common_countries]

    k = min(11, len(common_countries))

    # -------- Intensity --------
    print("--------- 处理 Intensity (左下角模式) ---------")

    Z_i, colors_i, info_i = compute_force_left_clustering(df_i, k)
    print(f"   [重心检测] 左侧权重: {info_i['left_weight']} | 右侧权重: {info_i['right_weight']}")

    if info_i["flipped"]:
        print("   >>> ⚠️ 检测到重心在右侧，正在强制执行全树镜像翻转 (Mirror Flip -> Left)...")
        print("   >>> 翻转完成，大数值已移至左下角。")
    else:
        print("   >>> ✅ 重心已在左侧，无需调整。")

    draw_heatmap(
        df_i, Z_i, colors_i,
        "4-1-independent_ForceLeft_INTENSITY.png",
        "Intensity",
        max_i
    )

    # -------- Breadth --------
    print("\n--------- 处理 Breadth (左下角模式) ---------")

    Z_b, colors_b, info_b = compute_force_left_clustering(df_b, k)
    print(f"   [重心检测] 左侧权重: {info_b['left_weight']} | 右侧权重: {info_b['right_weight']}")

    if info_b["flipped"]:
        print("   >>> ⚠️ 检测到重心在右侧，正在强制执行全树镜像翻转 (Mirror Flip -> Left)...")
        print("   >>> 翻转完成，大数值已移至左下角。")
    else:
        print("   >>> ✅ 重心已在左侧，无需调整。")

    draw_heatmap(
        df_b, Z_b, colors_b,
        "4-1-independent_ForceLeft_BREADTH.png",
        "Breadth",
        max_b
    )


if __name__ == "__main__":
    plot_independent_comparison(
        data_dir / "3-1-L2_Policy_Clustering_Breadth.csv",
        data_dir / "3-1-L2_Policy_Clustering_Intensity.csv"
    )


--------- 处理 Intensity (左下角模式) ---------
   [重心检测] 左侧权重: 8066 | 右侧权重: 7731
   >>> ✅ 重心已在左侧，无需调整。
✅ 保存成功: 4-1-independent_ForceLeft_INTENSITY.png

--------- 处理 Breadth (左下角模式) ---------
   [重心检测] 左侧权重: 6977 | 右侧权重: 8088
   >>> ⚠️ 检测到重心在右侧，正在强制执行全树镜像翻转 (Mirror Flip -> Left)...
   >>> 翻转完成，大数值已移至左下角。
✅ 保存成功: 4-1-independent_ForceLeft_BREADTH.png


#### 重心右下

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
import seaborn as sns
from scipy.cluster.hierarchy import linkage, fcluster, leaves_list
from pathlib import Path
import colorsys
import warnings

warnings.filterwarnings('ignore')

# ==========================================
# 1. 基础配置
# ==========================================
current_dir = Path.cwd()
data_dir = current_dir if (current_dir / "3-1-L2_Policy_Clustering_Breadth.csv").exists() else current_dir.parent / "data"


def get_times_black_font() -> FontProperties:
    return FontProperties(family='Times New Roman', size=26, weight='black')


T_BLACK = get_times_black_font()

plt.rcParams.update({
    'font.family': 'serif',
    'font.serif': ['Times New Roman'],
    'font.weight': 'black',
    'axes.labelweight': 'black',
    'axes.titleweight': 'black',
    'axes.unicode_minus': False,
    'figure.dpi': 300
})

sns.set_theme(style="white")
plt.rcParams['axes.axisbelow'] = False


def lighten_color_slightly(hex_color: str) -> str:
    r, g, b = plt.cm.colors.to_rgb(hex_color)
    h, l, s = colorsys.rgb_to_hls(r, g, b)
    return plt.cm.colors.to_hex(
        colorsys.hls_to_rgb(h, min(1.0, l + 0.12), s * 0.88)
    )


def get_palette(k: int) -> list:
    colors = [
        '#1f77b4', '#2ca02c', '#9467bd', '#8c564b',
        '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'
    ]
    return (colors * (k // len(colors) + 1))[:k]


# ==========================================
# 2. 数据处理
# ==========================================
def build_overlap_matrix(input_file: Path) -> tuple[pd.DataFrame, int]:
    df = pd.read_csv(input_file, encoding='utf-8-sig')

    if 'L2政策' not in df.columns and 'L2政策中文名' in df.columns:
        df.rename(columns={'L2政策中文名': 'L2政策'}, inplace=True)

    def safe_col(d: pd.DataFrame, name: str) -> pd.Series:
        return d[name].iloc[:, 0] if isinstance(d[name], pd.DataFrame) else d[name]

    df_clean = pd.DataFrame({
        'P': safe_col(df, 'L2政策').astype(str),
        'C': safe_col(df, '国家').astype(str),
        'G': safe_col(df, '聚类ID')
    }).drop_duplicates()

    policies = sorted(df_clean['P'].unique())
    countries = sorted(df_clean['C'].unique())

    res_map = {
        p: dict(zip(
            df_clean[df_clean['P'] == p]['C'],
            df_clean[df_clean['P'] == p]['G']
        ))
        for p in policies
    }

    n = len(countries)
    matrix = np.zeros((n, n), dtype=int)

    for i, c1 in enumerate(countries):
        for j in range(i, n):
            c2 = countries[j]
            common = sum(
                1 for p in policies
                if c1 in res_map[p]
                and c2 in res_map[p]
                and (res_map[p][c1] == res_map[p][c2] if i != j else True)
            )
            matrix[i, j] = matrix[j, i] = common

    return pd.DataFrame(matrix, index=countries, columns=countries), len(policies)


# ==========================================
# 3. 绘图
# ==========================================
def draw_heatmap(
    data_df: pd.DataFrame,
    Z_linkage,
    row_colors: pd.Series,
    filename: str,
    title: str,
    v_max: int
) -> None:

    size = max(18, len(data_df) * 0.72)

    g = sns.clustermap(
        data_df,
        row_linkage=Z_linkage,
        col_linkage=Z_linkage,
        cmap='RdYlBu_r',
        row_colors=row_colors,
        col_colors=row_colors,
        dendrogram_ratio=0.1,
        linewidths=0.5,
        figsize=(size, size),
        annot=True,
        fmt='d',
        vmin=0,
        vmax=v_max,
        annot_kws={'fontsize': 28, 'weight': 'black', 'fontproperties': T_BLACK},
        cbar_kws={'ticks': [0, 5, 10, 15, 20]},
        tree_kws={'linewidths': 6}
    )

    for lab in g.ax_heatmap.get_xticklabels():
        lab.set_fontproperties(T_BLACK)
        lab.set_rotation(45)
        lab.set_ha('right')

    for lab in g.ax_heatmap.get_yticklabels():
        lab.set_fontproperties(T_BLACK)

    save_path = data_dir / filename
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close('all')

    print(f"✅ 保存成功: {filename}")


# ==========================================
# 4. 强制右下角重心的聚类
# ==========================================
def compute_force_right_clustering(
    df: pd.DataFrame,
    k: int
) -> tuple[np.ndarray, pd.Series, dict]:

    Z = linkage(df.values, method='ward')
    weights = df.sum(axis=1).values

    # 局部：保证高权重在右子树（不打印）
    w_map = {i: v for i, v in enumerate(weights)}
    for i, (c1, c2, _, _) in enumerate(Z):
        c1, c2 = int(c1), int(c2)
        if w_map[c1] > w_map[c2]:
            Z[i, 0], Z[i, 1] = c2, c1
        w_map[len(weights) + i] = w_map[c1] + w_map[c2]

    # 全局重心检测
    leaves = leaves_list(Z)
    mid = len(leaves) // 2
    left_weight = int(weights[leaves[:mid]].sum())
    right_weight = int(weights[leaves[mid:]].sum())

    flipped = False
    if left_weight > right_weight:
        Z[:, [0, 1]] = Z[:, [1, 0]]
        flipped = True

    labels = fcluster(Z, t=k, criterion='maxclust') - 1
    palette = get_palette(k)
    row_colors = pd.Series(
        [lighten_color_slightly(palette[l % len(palette)]) for l in labels],
        index=df.index
    )

    info = {
        "left_weight": left_weight,
        "right_weight": right_weight,
        "flipped": flipped
    }

    return Z, row_colors, info


# ==========================================
# 5. 主流程
# ==========================================
def plot_independent_comparison(
    breadth_file: Path,
    intensity_file: Path
) -> None:

    df_b, max_b = build_overlap_matrix(breadth_file)
    df_i, max_i = build_overlap_matrix(intensity_file)

    common_countries = sorted(set(df_b.index) & set(df_i.index))
    df_b = df_b.loc[common_countries, common_countries]
    df_i = df_i.loc[common_countries, common_countries]

    k = min(11, len(common_countries))

    # -------- Intensity --------
    print("--------- 处理 Intensity (右下角模式) ---------")

    Z_i, colors_i, info_i = compute_force_right_clustering(df_i, k)
    print(f"   [重心检测] 左侧权重: {info_i['left_weight']} | 右侧权重: {info_i['right_weight']}")

    if info_i["flipped"]:
        print("   >>> ⚠️ 检测到重心在左侧，正在强制执行全树镜像翻转 (Mirror Flip -> Right)...")
        print("   >>> 翻转完成，大数值已移至右下角。")
    else:
        print("   >>> ✅ 重心已在右侧，无需调整。")

    draw_heatmap(
        df_i, Z_i, colors_i,
        "4-1-independent_ForceRight_INTENSITY.png",
        "Intensity",
        max_i
    )

    # -------- Breadth --------
    print("\n--------- 处理 Breadth (右下角模式) ---------")

    Z_b, colors_b, info_b = compute_force_right_clustering(df_b, k)
    print(f"   [重心检测] 左侧权重: {info_b['left_weight']} | 右侧权重: {info_b['right_weight']}")

    if info_b["flipped"]:
        print("   >>> ⚠️ 检测到重心在左侧，正在强制执行全树镜像翻转 (Mirror Flip -> Right)...")
        print("   >>> 翻转完成，大数值已移至右下角。")
    else:
        print("   >>> ✅ 重心已在右侧，无需调整。")

    draw_heatmap(
        df_b, Z_b, colors_b,
        "4-1-independent_ForceRight_BREADTH.png",
        "Breadth",
        max_b
    )


if __name__ == "__main__":
    plot_independent_comparison(
        data_dir / "3-1-L2_Policy_Clustering_Breadth.csv",
        data_dir / "3-1-L2_Policy_Clustering_Intensity.csv"
    )



--------- 处理 Intensity (右下角模式) ---------
   [重心检测] 左侧权重: 7357 | 右侧权重: 8440
   >>> ✅ 重心已在右侧，无需调整。
✅ 保存成功: 4-1-independent_ForceRight_INTENSITY.png

--------- 处理 Breadth (右下角模式) ---------
   [重心检测] 左侧权重: 7844 | 右侧权重: 7221
   >>> ⚠️ 检测到重心在左侧，正在强制执行全树镜像翻转 (Mirror Flip -> Right)...
   >>> 翻转完成，大数值已移至右下角。
✅ 保存成功: 4-1-independent_ForceRight_BREADTH.png


#### 不强制K值

In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
import seaborn as sns
from scipy.cluster.hierarchy import linkage, leaves_list
from pathlib import Path
import colorsys
import warnings

warnings.filterwarnings('ignore')

# ==========================================
# 1. 基础配置
# ==========================================
current_dir = Path.cwd()
data_dir = current_dir if (current_dir / "3-1-L2_Policy_Clustering_Breadth.csv").exists() else current_dir.parent / "data"


def get_times_black_font() -> FontProperties:
    return FontProperties(family='Times New Roman', size=26, weight='black')


T_BLACK = get_times_black_font()

plt.rcParams.update({
    'font.family': 'serif',
    'font.serif': ['Times New Roman'],
    'font.weight': 'black',
    'axes.labelweight': 'black',
    'axes.titleweight': 'black',
    'axes.unicode_minus': False,
    'figure.dpi': 300
})

sns.set_theme(style="white")
plt.rcParams['axes.axisbelow'] = False


def lighten_color_slightly(color) -> str:
    r, g, b = plt.cm.colors.to_rgb(color)
    h, l, s = colorsys.rgb_to_hls(r, g, b)
    return plt.cm.colors.to_hex(
        colorsys.hls_to_rgb(h, min(1.0, l + 0.12), s * 0.88)
    )


# ==========================================
# 2. 数据处理
# ==========================================
def build_overlap_matrix(input_file: Path) -> tuple[pd.DataFrame, int]:
    df = pd.read_csv(input_file, encoding='utf-8-sig')

    if 'L2政策' not in df.columns and 'L2政策中文名' in df.columns:
        df.rename(columns={'L2政策中文名': 'L2政策'}, inplace=True)

    def safe_col(d: pd.DataFrame, name: str) -> pd.Series:
        return d[name].iloc[:, 0] if isinstance(d[name], pd.DataFrame) else d[name]

    df_clean = pd.DataFrame({
        'P': safe_col(df, 'L2政策').astype(str),
        'C': safe_col(df, '国家').astype(str),
        'G': safe_col(df, '聚类ID')
    }).drop_duplicates()

    policies = sorted(df_clean['P'].unique())
    countries = sorted(df_clean['C'].unique())

    res_map = {
        p: dict(zip(
            df_clean[df_clean['P'] == p]['C'],
            df_clean[df_clean['P'] == p]['G']
        ))
        for p in policies
    }

    n = len(countries)
    matrix = np.zeros((n, n), dtype=int)

    for i, c1 in enumerate(countries):
        for j in range(i, n):
            c2 = countries[j]
            common = sum(
                1 for p in policies
                if c1 in res_map[p]
                and c2 in res_map[p]
                and (res_map[p][c1] == res_map[p][c2] if i != j else True)
            )
            matrix[i, j] = matrix[j, i] = common

    return pd.DataFrame(matrix, index=countries, columns=countries), len(policies)


# ==========================================
# 3. 绘图
# ==========================================
def draw_heatmap(
    data_df: pd.DataFrame,
    Z_linkage,
    row_colors: pd.Series,
    filename: str,
    title: str,
    v_max: int
) -> None:

    size = max(18, len(data_df) * 0.72)

    g = sns.clustermap(
        data_df,
        row_linkage=Z_linkage,
        col_linkage=Z_linkage,
        cmap='RdYlBu_r',
        row_colors=row_colors,
        col_colors=row_colors,
        dendrogram_ratio=0.1,
        linewidths=0.5,
        figsize=(size, size),
        annot=True,
        fmt='d',
        vmin=0,
        vmax=v_max,
        annot_kws={'fontsize': 28, 'weight': 'black', 'fontproperties': T_BLACK},
        tree_kws={'linewidths': 6}
    )

    save_path = data_dir / filename
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close('all')

    print(f"✅ 保存成功: {filename}")


# ==========================================
# 4. 左下角重心聚类（不强制 k）
# ==========================================
def compute_force_left_clustering(
    df: pd.DataFrame
) -> tuple[np.ndarray, pd.Series, dict]:

    Z = linkage(df.values, method='ward')
    weights = df.sum(axis=1).values

    # ---- 局部：保证大权重在左子树 ----
    w_map = {i: v for i, v in enumerate(weights)}
    for i, (c1, c2, _, _) in enumerate(Z):
        c1, c2 = int(c1), int(c2)
        if w_map[c1] < w_map[c2]:
            Z[i, 0], Z[i, 1] = c2, c1
        w_map[len(weights) + i] = w_map[c1] + w_map[c2]

    # ---- 全局：检测左右重心 ----
    leaves = leaves_list(Z)
    mid = len(leaves) // 2
    left_weight = int(weights[leaves[:mid]].sum())
    right_weight = int(weights[leaves[mid:]].sum())

    flipped = False
    if right_weight > left_weight:
        Z[:, [0, 1]] = Z[:, [1, 0]]
        flipped = True

    # ---- 颜色：按叶序渐变（不暗示聚类数） ----
    n = len(df)
    palette = sns.color_palette("husl", n)
    row_colors = pd.Series(
        [lighten_color_slightly(palette[i]) for i in range(n)],
        index=df.index[leaves]
    ).reindex(df.index)

    info = {
        "left_weight": left_weight,
        "right_weight": right_weight,
        "flipped": flipped
    }

    return Z, row_colors, info


# ==========================================
# 5. 主流程
# ==========================================
def plot_independent_comparison(
    breadth_file: Path,
    intensity_file: Path
) -> None:

    df_b, max_b = build_overlap_matrix(breadth_file)
    df_i, max_i = build_overlap_matrix(intensity_file)

    common_countries = sorted(set(df_b.index) & set(df_i.index))
    df_b = df_b.loc[common_countries, common_countries]
    df_i = df_i.loc[common_countries, common_countries]

    # -------- Intensity --------
    print("--------- 处理 Intensity (左下角模式 / 无 k) ---------")

    Z_i, colors_i, info_i = compute_force_left_clustering(df_i)
    print(f"   [重心检测] 左侧权重: {info_i['left_weight']} | 右侧权重: {info_i['right_weight']}")

    if info_i["flipped"]:
        print("   >>> ⚠️ 重心在右侧，已执行全树镜像翻转 → 左下角")
    else:
        print("   >>> ✅ 重心已在左侧，无需调整")

    draw_heatmap(
        df_i, Z_i, colors_i,
        "4-1-independent_ForceLeft_INTENSITY.png",
        "Intensity",
        max_i
    )

    # -------- Breadth --------
    print("\n--------- 处理 Breadth (左下角模式 / 无 k) ---------")

    Z_b, colors_b, info_b = compute_force_left_clustering(df_b)
    print(f"   [重心检测] 左侧权重: {info_b['left_weight']} | 右侧权重: {info_b['right_weight']}")

    if info_b["flipped"]:
        print("   >>> ⚠️ 重心在右侧，已执行全树镜像翻转 → 左下角")
    else:
        print("   >>> ✅ 重心已在左侧，无需调整")

    draw_heatmap(
        df_b, Z_b, colors_b,
        "4-1-independent_ForceLeft_BREADTH.png",
        "Breadth",
        max_b
    )


if __name__ == "__main__":
    plot_independent_comparison(
        data_dir / "3-1-L2_Policy_Clustering_Breadth.csv",
        data_dir / "3-1-L2_Policy_Clustering_Intensity.csv"
    )


--------- 处理 Intensity (左下角模式 / 无 k) ---------
   [重心检测] 左侧权重: 8066 | 右侧权重: 7731
   >>> ✅ 重心已在左侧，无需调整
✅ 保存成功: 4-1-independent_ForceLeft_INTENSITY.png

--------- 处理 Breadth (左下角模式 / 无 k) ---------
   [重心检测] 左侧权重: 6977 | 右侧权重: 8088
   >>> ⚠️ 重心在右侧，已执行全树镜像翻转 → 左下角
✅ 保存成功: 4-1-independent_ForceLeft_BREADTH.png
